In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from string import digits
import re
from sklearn.utils import shuffle
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input, Dense,Embedding
from keras.models import Model,load_model
# from keras.utils import plot_model
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
import pickle as pkl
import numpy as np

In [3]:
data_path = "/content/drive/MyDrive/pol.txt"

In [4]:
with open('/content/drive/MyDrive/pol.txt','r',encoding="utf8") as f:
  data = f.read()

In [5]:
# we need to clean the data
uncleaned_data_list = data.split('\n')
print(len(uncleaned_data_list))

uncleaned_data_list = uncleaned_data_list[:40465]
print(len(uncleaned_data_list))

english_word = []
polish_word = []
# cleaned_data_list = []
for word in uncleaned_data_list:
  english_word.append(word.split('\t')[:-1][0])
  polish_word.append(word.split('\t')[:-1][1])
  
print(len(english_word), len(polish_word))

40466
40465
40465 40465


In [6]:
language_data = pd.DataFrame(columns=['English','Polish'])
language_data['English'] = english_word
language_data['Polish'] = polish_word

# saving to csv
language_data.to_csv('language_data.csv', index=False)

# loading data from csv
language_data = pd.read_csv('language_data.csv')

In [7]:
lines= pd.read_table('language_data.csv', delimiter=",", names=['eng', 'pol'])

In [8]:
lines

,eng,pol
0,English,Polish
1,Go.,Idź.
2,Hi.,Cześć.
3,Run!,Uciekaj!
4,Run.,Biegnij.
...,...,...
40461,No matter how much you try to convince people ...,"Nieważne, jak bardzo usiłujesz przekonać ludzi..."
40462,A child who is a native speaker usually knows ...,Dziecko zwykle wie o swoim języku ojczystym rz...
40463,Since there are usually multiple websites on a...,Zwykle jest wiele stron internetowych na każdy...
40464,"If you want to sound like a native speaker, yo...","Jeśli chcesz mówić jak rodzimy użytkownik, mus..."


In [9]:


# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.pol=lines.pol.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.pol=lines.pol.apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation) # Set of all special characters

# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.pol=lines.pol.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.pol = lines.pol.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.pol=lines.pol.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.pol=lines.pol.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.eng = lines.eng.apply(lambda x : 'START_ '+ x + ' _END')

In [10]:
lines.eng

0                                      START_ english _END
1                                           START_ go _END
2                                           START_ hi _END
3                                          START_ run _END
4                                          START_ run _END
                               ...                        
40461    START_ no matter how much you try to convince ...
40462    START_ a child who is a native speaker usually...
40463    START_ since there are usually multiple websit...
40464    START_ if you want to sound like a native spea...
40465    START_ if someone who doesnt know your backgro...
Name: eng, Length: 40466, dtype: object

In [11]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of French 
all_polish_words=set()
for pol in lines.pol:
    for word in pol.split():
        if word not in all_polish_words:
            all_polish_words.add(word)

# Max Length of source sequence
lenght_list=[]
for l in lines.pol:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)

# Max Length of target sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)

input_words = sorted(list(all_polish_words))
target_words = sorted(list(all_eng_words))

# Calculate Vocab size for both source and target
num_encoder_tokens = len(all_polish_words)
num_encoder_tokens += 1
num_decoder_tokens = len(all_eng_words)
num_decoder_tokens += 1 # For zero padding

# Create word to token dictionary for both source and target
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

# Create token to word dictionary for both source and target
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [12]:

# Train - Test Split
X, y = lines.pol,lines.eng
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.01,random_state=101)
X_train.shape, X_test.shape

((40061,), (405,))

In [13]:

X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [14]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [15]:
latent_dim = 300

In [16]:

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [17]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [20]:

r = model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
312/312 [==============================] - 84s 199ms/step - loss: 0.8279 - accuracy: 0.1803 - val_loss: 0.6670 - val_accuracy: 0.2620
Epoch 2/50
312/312 [==============================] - 58s 187ms/step - loss: 0.6312 - accuracy: 0.2864 - val_loss: 0.5787 - val_accuracy: 0.3291
Epoch 3/50
312/312 [==============================] - 59s 189ms/step - loss: 0.5465 - accuracy: 0.3554 - val_loss: 0.5308 - val_accuracy: 0.3759
Epoch 4/50
312/312 [==============================] - 58s 186ms/step - loss: 0.4915 - accuracy: 0.4052 - val_loss: 0.4958 - val_accuracy: 0.4067
Epoch 5/50
312/312 [==============================] - 59s 189ms/step - loss: 0.4488 - accuracy: 0.4453 - val_loss: 0.4742 - val_accuracy: 0.4282
Epoch 6/50
312/312 [==============================] - 58s 187ms/step - loss: 0.4116 - accuracy: 0.4817 - val_loss: 0.4545 - val_accuracy: 0.4501
Epoch 7/50
312/312 [==============================] - 58s 184ms/step - loss: 0.3794 - accuracy: 0.5162 - val_loss: 0.4399 - val_ac

In [21]:
model.save("/content/drive/MyDrive/Pol-Eng Master/s2sPE.h5")
model.save("/content/drive/MyDrive/Pol-Eng Master/s2sPE",save_format='tf')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Pol-Eng Master/s2sPE/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Pol-Eng Master/s2sPE/assets


In [22]:
from tensorflow.keras.models import load_model

In [23]:
model = load_model("/content/drive/MyDrive/Pol-Eng Master/s2sPE.h5")

In [24]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [25]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [28]:
from nltk.translate.bleu_score import sentence_bleu
bleu_score = []
sent_len = []

In [29]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1
(test_gen)

<generator object generate_batch at 0x7f692f4c4bd0>

In [30]:
for k in range(0,10):
  print(k)
  (input_seq, actual_output), _ = next(test_gen)
  decoded_sentence = decode_sequence(input_seq)
  print('Input Polish sentence:', X_test[k:k+1].values[0])
  print('Actual English Translation:', y_test[k:k+1].values[0][6:-4])
  reference = [y_test[k:k+1].values[0][6:-4].split()]
  length = len(y_test[k:k+1].values[0][6:-4].split())
  sent_len.append(sent_len)
  print(reference)
  print('Predicted English Translation:', decoded_sentence[:-4])
  candidate = decoded_sentence[:-4].split()
  score = sentence_bleu(reference, candidate)
  bleu_score.append(score)

  print('BLEU score -> {}'.format(score))

0
Input Polish sentence: to nasz ojciec
Actual English Translation:  that is our father 
[['that', 'is', 'our', 'father']]
Predicted English Translation:  thats our father 
BLEU score -> 0.5444460596606694
1


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Input Polish sentence: ona robi wrażenie
Actual English Translation:  shes awesome 
[['shes', 'awesome']]
Predicted English Translation:  she is her a friend 
BLEU score -> 0
2
Input Polish sentence: chcę dla ciebie tego co najlepsze
Actual English Translation:  i want whats best for you 
[['i', 'want', 'whats', 'best', 'for', 'you']]
Predicted English Translation:  i want to do it for you say that 
BLEU score -> 0.5773502691896258
3
Input Polish sentence: oni cię nie chcą
Actual English Translation:  they dont want you 
[['they', 'dont', 'want', 'you']]
Predicted English Translation:  they dont want to do 
BLEU score -> 0.5623413251903491
4


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Input Polish sentence: gdyby tutaj teraz była mógłbym jej powiedzieć prawdę
Actual English Translation:  if she were here now i could tell her the truth 
[['if', 'she', 'were', 'here', 'now', 'i', 'could', 'tell', 'her', 'the', 'truth']]
Predicted English Translation:  if you were had the same i told you how to never 
BLEU score -> 0.7598356856515925
5


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Input Polish sentence: on był bardzo stary i schorowany
Actual English Translation:  he was very old and ill 
[['he', 'was', 'very', 'old', 'and', 'ill']]
Predicted English Translation:  he was married and old man was blue 
BLEU score -> 0.5169731539571706
6
Input Polish sentence: tylko ty możesz to zrobić
Actual English Translation:  youre the only one who can do this 
[['youre', 'the', 'only', 'one', 'who', 'can', 'do', 'this']]
Predicted English Translation:  you are just do that you 
BLEU score -> 0.45782273986766686
7
Input Polish sentence: wszyscy zachwycali się jej odwagą
Actual English Translation:  everyone marvelled at her courage 
[['everyone', 'marvelled', 'at', 'her', 'courage']]
Predicted English Translation:  everyone everyone want to cook 
BLEU score -> 0.668740304976422
8
Input Polish sentence: lubię słuchać podkastów
Actual English Translation:  i like to listen to podcasts 
[['i', 'like', 'to', 'listen', 'to', 'podcasts']]
Predicted English Translation:  i like to ca